In [1]:
library("MatrixEQTL")
library(tidyverse)
library(cqn)

── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.5.0     ✔ purrr   1.0.2
✔ tibble  3.2.1     ✔ dplyr   1.1.4
✔ tidyr   1.3.1     ✔ stringr 1.5.1
✔ readr   2.1.4     ✔ forcats 1.0.0
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: mclust

Package 'mclust' version 6.1
Type 'citation("mclust")' for citing this R package in publications.


Attaching package: ‘mclust’


The following object is masked from ‘package:purrr’:

    map


Loading required package: nor1mix

Loading required package: preprocessCore

Loading required package: splines

Loading required package: quantreg

Loading required package: SparseM


Attaching package: ‘SparseM’


The following object is masked from ‘package:b

# Get GC Bias 

##### This only needs to be run once, can be used across cell types

In [2]:
library(Seurat)

Loading required package: SeuratObject

Loading required package: SeuratObject

Loading required package: sp

Loading required package: sp


Attaching package: ‘SeuratObject’



Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


The following object is masked from ‘package:base’:

    intersect




In [3]:
#Load in Seurat ATAC object
atac <- readRDS('/data/clustering/221125_28sample_combined.noDoublets.QCfilters.ATAC_peaks.FINAL_v2.w_mergedPeaks_SM.rds')
DefaultAssay(atac) <- 'ATAC_CTpeaks'

atac

Loading required package: Signac

Loading required package: Signac



An object of class Seurat 
543177 features across 174819 samples within 4 assays 
Active assay: ATAC_CTpeaks (263223 features, 0 variable features)
 2 layers present: counts, data
 3 other assays present: RNA, ATAC, SCT
 7 dimensional reductions calculated: pca, harmony.rna, umap.rna, lsi, harmony.atac, umap.atac, umap.wnn

An object of class Seurat 
543177 features across 174819 samples within 4 assays 
Active assay: ATAC_CTpeaks (263223 features, 0 variable features)
 2 layers present: counts, data
 3 other assays present: RNA, ATAC, SCT
 7 dimensional reductions calculated: pca, harmony.rna, umap.rna, lsi, harmony.atac, umap.atac, umap.wnn

In [4]:
atac[['RNA']] <- NULL
atac[['SCT']] <- NULL
atac[['ATAC']] <- NULL

atac
gc()

An object of class Seurat 
263223 features across 174819 samples within 1 assay 
Active assay: ATAC_CTpeaks (263223 features, 0 variable features)
 2 layers present: counts, data
 7 dimensional reductions calculated: pca, harmony.rna, umap.rna, lsi, harmony.atac, umap.atac, umap.wnn

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,8565735,457.5,13523501,722.3,13523501,722.3
Vcells,4388165144,33479.1,12856643525,98088.5,11290315012,86138.3


An object of class Seurat 
263223 features across 174819 samples within 1 assay 
Active assay: ATAC_CTpeaks (263223 features, 0 variable features)
 2 layers present: counts, data
 7 dimensional reductions calculated: pca, harmony.rna, umap.rna, lsi, harmony.atac, umap.atac, umap.wnn

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,8565735,457.5,13523501,722.3,13523501,722.3
Vcells,4388165144,33479.1,12856643525,98088.5,11290315012,86138.3


In [5]:
#Prep inputs for SummarizedExperiment

#Extract matrix from Seurat object
sc.data <- GetAssayData(atac,slot='data', assay="ATAC_CTpeaks")
sc.data.stored <- sc.data

# Extract peak locations and reformat into GRanges object
bed = str_split_fixed(rownames(sc.data), "\\-", 3) #This may need to be modified depending on your peak naming convention

gr= GRanges(seqnames = bed[,1], ranges = IRanges(start = as.numeric(bed[,2]), end = as.numeric(bed[,3])))

Warning message:
“The `slot` argument of `GetAssayData()` is deprecated as of SeuratObject 5.0.0.
ℹ Please use the `layer` argument instead.”


ERROR: Error in GRanges(seqnames = bed[, 1], ranges = IRanges(start = as.numeric(bed[, : could not find function "GRanges"


Warning message:
“The `slot` argument of `GetAssayData()` is deprecated as of SeuratObject 5.0.0.
ℹ Please use the `layer` argument instead.”


ERROR: Error in GRanges(seqnames = bed[, 1], ranges = IRanges(start = as.numeric(bed[, : could not find function "GRanges"


In [ ]:
library(BSgenome.Hsapiens.UCSC.hg38)
library(chromVAR)

In [ ]:
#Use the matrix and GRanges to make a SummarizedExperiment to put into chromvar and add metadata

fragment.counts <- SummarizedExperiment(assays=list(counts=sc.data), rowRanges=gr)
fragment.counts.stored <- fragment.counts

fragment.counts <- addGCBias(fragment.counts, genome=BSgenome.Hsapiens.UCSC.hg38)
fragment.counts

In [ ]:
bias_data <- rowData(fragment.counts)
bias_data

In [ ]:
write.table(bias_data, "/path/to/caQTL/stuff/atac_cell_type_matrices/240402_peak_bias.tsv", 
           sep='\t', quote=FALSE, col.names=TRUE, row.names=TRUE)

# CQN Normalization

In [ ]:
cell_types <- c('acinar','alpha','bulk','delta','ductal','gamma')
names(cell_types) <- cell_types

In [ ]:
samples <- list()

for (cell_type in cell_types) {
    samples[[cell_type]] <- unlist(read.table(paste0('/path/to/caQTL/stuff/atac_cell_type_matrices/samples.', cell_type, '.csv'), sep=','))
}

summary(samples)
samples

In [4]:
counts <- list()
bias_data <- read.table("/path/to/caQTL/stuff/atac_cell_type_matrices/240402_peak_bias.tsv")
rownames(bias_data) <- stringr::str_replace(rownames(bias_data), "-",":")

for (cell_type in cell_types) {
    counts[[cell_type]] <- read.table(paste0('/path/to/caQTL/stuff/atac_cell_type_matrices/counts.', cell_type, '.txt'), row.names=1, header=T)
}

summary(counts)
dim(counts[['acinar']])
head(counts[['acinar']])

       Length Class      Mode
acinar 27     data.frame list
alpha  27     data.frame list
bulk   27     data.frame list
delta  26     data.frame list
ductal 26     data.frame list
gamma  26     data.frame list

[1] 116294     27

,X55,X153,X16,X115,X33,X51,X193,X18,X19,X53,⋯,X17,X18.1,X24,X54,X73,X314,X253,X48,X79,X241
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
chr1:99525-99851,4,4,4,6,0,0,16,0,1,9,⋯,2,2,7,0,12,66,40,13,34,28
chr1:102751-103113,4,3,2,30,5,2,8,3,4,10,⋯,2,5,5,3,8,39,32,6,13,9
chr1:127650-127914,2,3,4,10,2,5,5,1,5,3,⋯,0,3,7,0,10,13,8,7,9,5
chr1:180676-181929,48,184,18,87,44,25,205,25,36,58,⋯,10,22,23,57,43,230,291,31,92,215
chr1:184056-184566,7,26,0,18,9,10,23,7,5,14,⋯,2,3,1,8,6,44,35,7,16,44
chr1:190880-191998,5,19,2,23,7,3,14,2,2,7,⋯,0,2,6,1,1,21,10,5,2,13


In [5]:
for (cell_type in cell_types) {
    print(cell_type)
    print(dim(counts[[cell_type]]))
}

[1] "acinar"
[1] 116294     27
[1] "alpha"
[1] 197653     27
[1] "bulk"
[1] 262915     27
[1] "delta"
[1] 89174    26
[1] "ductal"
[1] 51209    26
[1] "gamma"
[1] 64321    26


In [ ]:
head(rownames(bias_data))

In [ ]:
for (cell_type in cell_types) {
    print(cell_type)
    print(paste0("Bias matches in counts for ", cell_type, ": ", sum(rownames(bias_data) %in% rownames(counts[[cell_type]]))))
    print(paste0("Length of bias data: ", length(rownames(bias_data))))
    
    print(paste0("Counts matches in bias for ", cell_type, ": ", sum(rownames(counts[[cell_type]]) %in% rownames(bias_data))))
    print(paste0("Features in counts for ", cell_type, ": ", length(rownames(counts[[cell_type]]))))
    colnames(counts[[cell_type]]) <- samples[[cell_type]]
}

In [ ]:
bias_data_to_join <- tibble::rownames_to_column(bias_data, var="peak")
#bias_data_to_join

bias_data_counts <- list()
counts_ordered <- list()

for (cell_type in cell_types) {
    bias_data_counts[[cell_type]] <- dplyr::filter(bias_data_to_join, peak %in% rownames(counts[[cell_type]]))
    bias_data_counts[[cell_type]] <- dplyr::arrange(bias_data_counts[[cell_type]], peak)
    
    counts_ordered[[cell_type]] <- dplyr::arrange(counts[[cell_type]], rownames(counts[[cell_type]]))
}

summary(bias_data_counts)
dim(bias_data_counts[['acinar']])
head(bias_data_counts[['acinar']])

summary(counts_ordered)
dim(counts_ordered[['acinar']])
head(counts_ordered[['acinar']])

In [ ]:
peak_start_end <- list()

for (cell_type in cell_types) {
    peak_start_end[[cell_type]] <- stringr::str_split(stringr::str_split(bias_data_counts[[cell_type]]$peak, ":", simplify=T)[,2], "-", simplify=TRUE)
    peak_start_end[[cell_type]] <- data.frame(peak_start_end[[cell_type]])
    rownames(peak_start_end[[cell_type]]) <- bias_data_counts[[cell_type]]$peak
    
    peak_start_end[[cell_type]]$length <- as.integer(peak_start_end[[cell_type]][,2]) - as.integer(peak_start_end[[cell_type]][,1])
}

summary(peak_start_end)
dim(peak_start_end[['acinar']])
head(peak_start_end[['acinar']])

In [ ]:
for (cell_type in cell_types) {
    print(cell_type)
    print(length(rownames(counts_ordered[[cell_type]])))
    print(sum(rownames(counts_ordered[[cell_type]]) == bias_data_counts[[cell_type]]$peak))
    print(sum(rownames(counts_ordered[[cell_type]]) == rownames(peak_start_end[[cell_type]])))
}

In [ ]:
cqn <- list()

for (cell_type in cell_types) {
    print(Sys.time())
    cqn[[cell_type]] <- cqn(counts_ordered[[cell_type]], x=bias_data_counts[[cell_type]]$bias, lengths=peak_start_end[[cell_type]]$length)
    print(Sys.time())
}


summary(cqn)
dim(cqn[['acinar']])
head(cqn[['acinar']])

In [ ]:
cqn.normalized <- list()

for (cell_type in cell_types) {
    cqn.normalized[[cell_type]] <- cqn[[cell_type]]$y + cqn[[cell_type]]$offset
    write.table(cqn.normalized[[cell_type]], paste0('/path/to/caQTL/stuff/atac_cell_type_matrices/counts.', cell_type, '_cqn.txt'),
            row.names=TRUE, col.names=TRUE, quote=FALSE, sep='\t')
}

summary(cqn.normalized)
dim(cqn.normalized[['acinar']])
head(cqn.normalized[['acinar']])

In [ ]:
summary(cqn.normalized)
dim(cqn.normalized[['acinar']])
head(cqn.normalized[['acinar']])

# Prepare Inputs for each Feature

In [2]:
cell_types <- c('acinar','alpha','bulk','delta','ductal','gamma')
names(cell_types) <- cell_types

cqn.normalized <- list()

for (cell_type in cell_types) {
    cqn.normalized[[cell_type]] <- read.table(paste0('/path/to/caQTL/stuff/atac_cell_type_matrices/counts.', cell_type, '_cqn.txt'),
            header=TRUE, sep='\t')
}

summary(cqn.normalized)
dim(cqn.normalized[['acinar']])
head(cqn.normalized[['acinar']])

       Length Class      Mode
acinar 27     data.frame list
alpha  27     data.frame list
bulk   27     data.frame list
delta  26     data.frame list
ductal 26     data.frame list
gamma  26     data.frame list

[1] 116294     27

,R207,R217,R218,R221,R223,R226,R228,R234,R237,R238,⋯,R317,R319,R325,R326,R327,R332,R353,R354,R362,R363
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr10:100006062-100006793,3.4467132,3.142236,3.9446266,3.970342,3.738282,2.800763,2.9410033,1.6487903,3.776511,2.6744257,⋯,2.6212862,3.238641,2.123923,3.2006505,3.9960024,3.3042398,2.6694148,1.772715,3.4048370,2.5675675
chr10:100009188-100010635,4.6175720,4.638407,4.7401527,4.432949,4.824566,4.742232,4.5656178,4.4236984,4.471096,4.4049415,⋯,4.1805709,4.680129,4.509128,4.6722978,4.8130093,4.6775736,4.3982980,4.671813,4.5517049,4.6084907
chr10:100019905-100021214,0.8810361,1.073495,0.9499951,1.183826,1.918379,1.026178,0.8810361,0.9682674,1.203189,0.5881409,⋯,0.5881409,1.527011,1.153134,0.7683525,0.4935815,0.4935815,0.5881409,-1.012646,0.5881409,0.4316394
chr10:100037925-100038813,2.7627797,2.961770,1.9227798,2.936885,1.824817,2.450239,3.3468372,3.2155268,3.556330,2.0267927,⋯,2.7899830,2.749991,3.407186,3.1480622,3.3197710,3.1122357,2.2234132,3.415192,2.5115755,2.7739250
chr10:100043372-100044999,2.7495872,2.845891,2.1860520,3.502456,2.431333,2.467006,3.1874404,2.8202359,3.101875,3.0746209,⋯,2.1122082,1.808400,2.929221,3.4886769,3.5908965,3.7967946,3.4372894,2.880409,2.8017788,2.9036357
chr10:100046018-100046622,2.9812697,3.194533,3.0645726,3.581122,2.454713,2.825392,2.9515617,2.6825529,3.497613,2.6019127,⋯,1.9153333,1.194297,3.818690,3.3594470,3.1334339,3.1999666,2.9348463,3.745594,2.6258437,3.2815614


In [3]:
for (cell_type in cell_types) {
    print(cell_type)
    print(dim(cqn.normalized[[cell_type]]))
}

[1] "acinar"
[1] 116294     27
[1] "alpha"
[1] 197653     27
[1] "bulk"
[1] 262915     27
[1] "delta"
[1] 89174    26
[1] "ductal"
[1] 51209    26
[1] "gamma"
[1] 64321    26


In [13]:
samples <- list()

for (cell_type in cell_types) {
    samples[[cell_type]] <- unlist(read.table(paste0('/path/to/caQTL/stuff/atac_cell_type_matrices/samples.', cell_type, '.csv'), sep=','))
}

summary(samples)
samples

       Length Class  Mode     
acinar 27     -none- character
alpha  27     -none- character
bulk   27     -none- character
delta  26     -none- character
ductal 26     -none- character
gamma  26     -none- character

$acinar
    x1     x2     x3     x4     x5     x6     x7     x8     x9    x10    x11 
"R207" "R217" "R218" "R221" "R223" "R226" "R228" "R234" "R237" "R238" "R246" 
   x12    x13    x14    x15    x16    x17    x18    x19    x20    x21    x22 
"R247" "R275" "R284" "R290" "R292" "R316" "R317" "R319" "R325" "R326" "R327" 
   x23    x24    x25    x26    x27 
"R332" "R353" "R354" "R362" "R363" 

$alpha
    x1     x2     x3     x4     x5     x6     x7     x8     x9    x10    x11 
"R207" "R217" "R218" "R221" "R223" "R226" "R228" "R234" "R237" "R238" "R246" 
   x12    x13    x14    x15    x16    x17    x18    x19    x20    x21    x22 
"R247" "R275" "R284" "R290" "R292" "R316" "R317" "R319" "R325" "R326" "R327" 
   x23    x24    x25    x26    x27 
"R332" "R353" "R354" "R362" "R363" 

$bulk
    x1     x2     x3     x4     x5     x6     x7     x8     x9    x10    x11 
"R207" "R217" "R218" "R221" "R223" "R226" "R228" "R234" "R237" "R238" "R246" 
   x12    x13    x14    x15    x16    x17    x18    x19    x20    x21    x22 
"R247" "R275" "R284" "R290" "R292" "R316" "R317" "R319" "R325" "R326" "R327" 
   x23    x24    x25    x26    x27 
"R332" "R353" "R354" "R362" "R363" 

$delta
    x1     x2     x3     x4     x5     x6     x7     x8     x9    x10    x11 
"R207" "R217" "R218" "R221" "R223" "R226" "R228" "R234" "R237" "R238" "R246" 
   x12    x13    x14    x15    x16    x17    x18    x19    x20    x21    x22 
"R247" "R275" "R284" "R290" "R292" "R316" "R317" "R319" "R326" "R327" "R332" 
   x23    x24    x25    x26 
"R353" "R354" "R362" "R363" 

$ductal
    x1     x2     x3     x4     x5     x6     x7     x8     x9    x10    x11 
"R207" "R217" "R218" "R221" "R223" "R228" "R234" "R237" "R238" "R246" "R247" 
   x12    x13    x14    x15    x16    x17    x18    x19    x20    x21    x22 
"R275" "R284" "R290" "R292" "R316" "R317" "R319" "R325" "R326" "R327" "R332" 
   x23    x24    x25    x26 
"R353" "R354" "R362" "R363" 

$gamma
    x1     x2     x3     x4     x5     x6     x7     x8     x9    x10    x11 
"R207" "R217" "R218" "R221" "R223" "R226" "R228" "R234" "R237" "R238" "R246" 
   x12    x13    x14    x15    x16    x17    x18    x19    x20    x21    x22 
"R275" "R284" "R290" "R292" "R316" "R317" "R319" "R325" "R326" "R327" "R332" 
   x23    x24    x25    x26 
"R353" "R354" "R362" "R363"

In [14]:
hannah_GWAS_peaks <- read.table('/references/hg38_GWAS/lead_variants/final_peaks_mergedPeak.overlap_all_lead_variants_100kb_windows.wINFO.bed')
hannah_GWAS_peaks

V1,V2,V3,V4,V5,V6,V7,V8,V9
<chr>,<int>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<chr>
chr1,3139257,3140991,chr1,3140723,3240723,rs12142172,FG,MAGIC_Glycemic
chr1,3141294,3141578,chr1,3140723,3240723,rs12142172,FG,MAGIC_Glycemic
chr1,3144228,3144677,chr1,3140723,3240723,rs12142172,FG,MAGIC_Glycemic
chr1,3145672,3145886,chr1,3140723,3240723,rs12142172,FG,MAGIC_Glycemic
chr1,3152654,3152854,chr1,3140723,3240723,rs12142172,FG,MAGIC_Glycemic
chr1,3155063,3156322,chr1,3140723,3240723,rs12142172,FG,MAGIC_Glycemic
chr1,3156716,3157169,chr1,3140723,3240723,rs12142172,FG,MAGIC_Glycemic
chr1,3162149,3162863,chr1,3140723,3240723,rs12142172,FG,MAGIC_Glycemic
chr1,3164682,3166139,chr1,3140723,3240723,rs12142172,FG,MAGIC_Glycemic


In [15]:
hannah_GWAS_peaks$feature <- paste0(hannah_GWAS_peaks$V1,":",hannah_GWAS_peaks$V2,"-",hannah_GWAS_peaks$V3)

length(unique(hannah_GWAS_peaks$feature))
head(hannah_GWAS_peaks)

[1] 10266

,V1,V2,V3,V4,V5,V6,V7,V8,V9,feature
,<chr>,<int>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
1,chr1,3139257,3140991,chr1,3140723,3240723,rs12142172,FG,MAGIC_Glycemic,chr1:3139257-3140991
2,chr1,3141294,3141578,chr1,3140723,3240723,rs12142172,FG,MAGIC_Glycemic,chr1:3141294-3141578
3,chr1,3144228,3144677,chr1,3140723,3240723,rs12142172,FG,MAGIC_Glycemic,chr1:3144228-3144677
4,chr1,3145672,3145886,chr1,3140723,3240723,rs12142172,FG,MAGIC_Glycemic,chr1:3145672-3145886
5,chr1,3152654,3152854,chr1,3140723,3240723,rs12142172,FG,MAGIC_Glycemic,chr1:3152654-3152854
6,chr1,3155063,3156322,chr1,3140723,3240723,rs12142172,FG,MAGIC_Glycemic,chr1:3155063-3156322


In [16]:
features_to_run <- unique(hannah_GWAS_peaks$feature)

In [17]:
for (cell_type in cell_types) {
    print(cell_type)
    print(sum(features_to_run %in% rownames(cqn.normalized[[cell_type]])))
}

[1] "acinar"
[1] 5209
[1] "alpha"
[1] 8294
[1] "bulk"
[1] 10262
[1] "delta"
[1] 4220
[1] "ductal"
[1] 2644
[1] "gamma"
[1] 3169


In [18]:
features_to_run_ct <- list()

for (cell_type in cell_types) {
    features_to_run_ct[[cell_type]] <- features_to_run[features_to_run %in% rownames(cqn.normalized[[cell_type]])]
}

summary(features_to_run_ct)
length(features_to_run_ct[['acinar']])
head(features_to_run_ct[['acinar']])

       Length Class  Mode     
acinar  5209  -none- character
alpha   8294  -none- character
bulk   10262  -none- character
delta   4220  -none- character
ductal  2644  -none- character
gamma   3169  -none- character

[1] 5209

[1] "chr1:3139257-3140991" "chr1:3144228-3144677" "chr1:3162149-3162863"
[4] "chr1:3164682-3166139" "chr1:3169726-3171629" "chr1:3173026-3173397"

In [19]:
genotypes = read.table("/path/to/caQTL/stuff/caQTLs_rasqual/genotypes/All_tested_genotypes_run1.tsv", header=T)

dim(genotypes)
head(genotypes)

[1] 11716077       27

,R207,R217,R218,R221,R223,R226,R228,R234,R237,R238,⋯,R317,R319,R325,R326,R327,R332,R353,R354,R362,R363
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
chr1:709040:T:C,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
chr1:727242:G:A,0,1,0,0,0,0,0,0,0,0,⋯,0,0,1,0,0,1,0,1,0,1
chr1:732994:G:A,0,0,0,0,0,0,0,0,1,0,⋯,0,0,0,0,0,0,0,0,0,0
chr1:752833:T:G,0,1,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,1,0,0,0,0
chr1:758351:A:G,0,1,0,0,0,0,0,0,0,0,⋯,0,0,1,0,0,1,0,1,0,1
chr1:758443:G:C,0,1,0,0,0,0,0,0,0,0,⋯,0,0,1,0,0,1,0,1,0,1


In [20]:
all_rasqual_results <- list()

for (cell_type in cell_types) {
    all_rasqual_results[[cell_type]] <- read.table(paste0('/path/to/caQTL/stuff/caQTLs_rasqual/',cell_type, '/run1/Results_run1_all.tsv'), header=T, stringsAsFactors = F)
}

summary(all_rasqual_results)
length(all_rasqual_results[['acinar']])
head(all_rasqual_results[['acinar']])

       Length Class      Mode
acinar 30     data.frame list
alpha  30     data.frame list
bulk   30     data.frame list
delta  30     data.frame list
ductal 30     data.frame list
gamma  30     data.frame list

[1] 30

,Feature,rsID,Chromosome,position,Ref,Alt,Af,HWEChi_square,IA,Log10_qval,⋯,ties,Log_likelihood_H0,Convergence_status,r2_fSNPs,r2_rSNP,P_VAl,dist,flag_fdr10,flag_fdr05,flag_fdr01
,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<lgl>,<lgl>,<lgl>
1,chr6:32622377-32623439,chr6:32620608:T:A,chr6,32620608,T,A,0.425926,2.231991,0.998678,-67.03611,⋯,32620608,156.3965,0,0.991233,0.824496,8.976233e-71,-2300,TRUE,TRUE,TRUE
2,chr6:32622377-32623439,chr6:32620292:C:T,chr6,32620292,C,T,0.481481,3.033058,0.997176,-67.03611,⋯,32620608,155.1102,0,0.991113,0.766739,3.415022e-70,-2616,TRUE,TRUE,TRUE
3,chr6:32622377-32623439,chr6:32620069:C:T,chr6,32620069,C,T,0.481481,3.033058,0.997249,-67.03611,⋯,32620608,155.1102,0,0.991113,0.766739,3.415022e-70,-2839,TRUE,TRUE,TRUE
4,chr6:32622377-32623439,chr6:32620254:G:A,chr6,32620254,G,A,0.481481,3.033058,0.997319,-67.03611,⋯,32620608,155.0975,0,0.991113,0.767647,3.459474e-70,-2654,TRUE,TRUE,TRUE
5,chr6:32622377-32623439,chr6:32623176:T:C,chr6,32623176,T,C,0.462963,1.913385,0.999952,-66.25852,⋯,32620608,152.2692,0,0.991231,0.777224,5.637594e-69,268,TRUE,TRUE,TRUE
6,chr6:32622377-32623439,chr6:32623122:G:T,chr6,32623122,G,T,0.462963,1.913385,0.999952,-66.25852,⋯,32620608,152.2670,0,0.987012,0.777217,5.650015e-69,214,TRUE,TRUE,TRUE


In [21]:
for (cell_type in cell_types) {
    print(cell_type)
    print(sum(features_to_run_ct[[cell_type]] %in% all_rasqual_results[[cell_type]]$Feature))
}


[1] "acinar"
[1] 5153
[1] "alpha"
[1] 8207
[1] "bulk"
[1] 10156
[1] "delta"
[1] 4178
[1] "ductal"
[1] 2616
[1] "gamma"
[1] 3135


In [22]:
for (cell_type in cell_types) {
    features_to_run_ct[[cell_type]] <- features_to_run_ct[[cell_type]][features_to_run_ct[[cell_type]] %in% all_rasqual_results[[cell_type]]$Feature]
}

summary(features_to_run_ct)
length(features_to_run_ct[['acinar']])
head(features_to_run_ct[['acinar']])

       Length Class  Mode     
acinar  5153  -none- character
alpha   8207  -none- character
bulk   10156  -none- character
delta   4178  -none- character
ductal  2616  -none- character
gamma   3135  -none- character

[1] 5153

[1] "chr1:3139257-3140991" "chr1:3144228-3144677" "chr1:3162149-3162863"
[4] "chr1:3164682-3166139" "chr1:3169726-3171629" "chr1:3173026-3173397"

In [23]:
covariates <- list()

for (cell_type in cell_types) {
    covariates[[cell_type]] <- data.table::transpose(read.table(paste0("/path/to/caQTL/stuff/atac_cell_type_matrices/covariates_full.", cell_type, ".txt"))) 
    colnames(covariates[[cell_type]]) <- samples[[cell_type]]
    covariates[[cell_type]]$id <- c("Size",paste0('PC', 1:(nrow(covariates[[cell_type]]) - 7)),"Age","Sex","gPC1","gPC2","gPC3","gPC4")
    covariates[[cell_type]] <- select(covariates[[cell_type]], id, everything())
    
    write.table(covariates[[cell_type]],
            paste0("/path/to/caQTL/stuff/gwas_overlap/Matrix_QTL_Inputs/Covariates.",cell_type,".tsv"), 
            col.names=T, quote=F, row.names=F, sep="\t")
}

summary(covariates)
length(covariates[['acinar']])
head(covariates[['acinar']])

       Length Class      Mode
acinar 28     data.frame list
alpha  28     data.frame list
bulk   28     data.frame list
delta  27     data.frame list
ductal 27     data.frame list
gamma  27     data.frame list

[1] 28

,id,R207,R217,R218,R221,R223,R226,R228,R234,R237,⋯,R317,R319,R325,R326,R327,R332,R353,R354,R362,R363
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Size,15.113076432,16.10377226,13.9167757,16.27246551,14.35899813,15.400460267,16.391986916,14.48901100,14.64497724,⋯,13.906778944,14.75675221,14.72580173,14.902660066,15.20870382,16.84745238,16.39559412,15.05531594,15.72512254,16.89371422
2,PC1,0.093237288,0.14565368,-0.5053538,-0.27237484,-0.17743784,0.184273280,0.012507245,0.10794325,-0.07721857,⋯,-0.109088971,0.25454870,0.03534093,0.001572438,-0.10953926,0.14294036,0.01378234,-0.01648860,0.22435679,0.17398102
3,PC2,-0.092460256,-0.08653872,0.4983219,-0.01397980,-0.12149037,-0.046431175,-0.044184875,-0.05146184,0.04390587,⋯,0.205224087,0.06838480,-0.12728936,0.021789392,-0.06364223,-0.05848616,-0.17806895,-0.11720129,-0.01392241,-0.09731979
4,PC3,0.002211525,-0.01152765,-0.1071077,-0.09424527,-0.03320605,-0.001569767,-0.010176376,-0.03393564,0.02458613,⋯,0.905451801,0.14490448,-0.06340692,-0.002300619,-0.04788104,-0.01245775,-0.07085165,-0.04215793,0.05668995,-0.02970293
5,PC4,-0.082938968,-0.11598453,-0.6338968,0.15533158,0.09021985,-0.160490153,-0.003238948,-0.08693324,0.05444152,⋯,0.202883485,-0.21563293,0.03535354,-0.055153233,0.09589441,-0.07274858,0.03635884,0.04451831,-0.16909166,-0.08535469
6,PC5,0.062077263,-0.11130179,0.0338984,-0.19682544,-0.54534759,-0.067908077,0.012588419,-0.01267762,0.07245659,⋯,-0.004028385,-0.01838241,-0.17280930,0.176645443,0.08792409,0.03494837,0.20369176,0.34971687,0.16565292,0.01118753


# Get SNPs

In [24]:
snp_details <- str_split(rownames(genotypes), ":", simplify=T)
colnames(snp_details) <- c('chr','pos','ref','alt')

snp_details

genotypes_detailed <- cbind(genotypes,snp_details)
genotypes_detailed$pos <- as.numeric(genotypes_detailed$pos)

dim(genotypes_detailed)
head(genotypes_detailed)

chr,pos,ref,alt
chr1,709040,T,C
chr1,727242,G,A
chr1,732994,G,A
chr1,752833,T,G
chr1,758351,A,G
chr1,758443,G,C
chr1,763097,C,T
chr1,763107,A,G
chr1,767578,T,C
chr1,771030,C,A


[1] 11716077       31

,R207,R217,R218,R221,R223,R226,R228,R234,R237,R238,⋯,R327,R332,R353,R354,R362,R363,chr,pos,ref,alt
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<dbl>,<chr>,<chr>
chr1:709040:T:C,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,chr1,709040,T,C
chr1:727242:G:A,0,1,0,0,0,0,0,0,0,0,⋯,0,1,0,1,0,1,chr1,727242,G,A
chr1:732994:G:A,0,0,0,0,0,0,0,0,1,0,⋯,0,0,0,0,0,0,chr1,732994,G,A
chr1:752833:T:G,0,1,0,0,0,0,0,0,0,0,⋯,0,1,0,0,0,0,chr1,752833,T,G
chr1:758351:A:G,0,1,0,0,0,0,0,0,0,0,⋯,0,1,0,1,0,1,chr1,758351,A,G
chr1:758443:G:C,0,1,0,0,0,0,0,0,0,0,⋯,0,1,0,1,0,1,chr1,758443,G,C


In [33]:
cell_types[4:6]

delta   ductal    gamma 
 "delta" "ductal"  "gamma"

In [34]:
input_QTL_dir <- '/path/to/caQTL/stuff/gwas_overlap/Matrix_QTL_Inputs'
#dir.create(input_QTL_dir)

snp_details <- str_split(rownames(genotypes), ":", simplify=T)
colnames(snp_details) <- c('chr','pos','ref','alt')

genotypes_detailed <- cbind(genotypes,snp_details)
genotypes_detailed$pos <- as.numeric(genotypes_detailed$pos)

#for (cell_type in cell_types) {
for (cell_type in cell_types[4:6]) {
    ct_dir <- paste0(input_QTL_dir,"/",cell_type)
    dir.create(ct_dir, showWarnings = FALSE)
    cqn.normalized[[cell_type]] <- data.frame(cqn.normalized[[cell_type]])
    
    for (feat in features_to_run_ct[[cell_type]]) {
    #for (feat in 'chr18:12776439-12778058') {
        feat_details <- str_split(feat, "[:-]", simplify=T)
        feat_chr <- feat_details[1]
        start <- as.numeric(feat_details[2]) - 100000
        end <- as.numeric(feat_details[3]) + 100000
        
        genotypes_filt <- genotypes[rownames(filter(genotypes_detailed, chr==feat_chr & pos >= start & pos <= end)),]
        genotypes_filt <- genotypes_filt[(rowSums(genotypes_filt) > (ncol(genotypes_filt) * 2 * .05)) &
                                         (rowSums(genotypes_filt) < (ncol(genotypes_filt) * 2 * .95)),]
        
        genotypes_filt <- genotypes_filt[samples[[cell_type]]]
        
        write.table(rownames_to_column(genotypes_filt, var='snpid'), 
                paste0(ct_dir, "/", feat, "_Genotypes_100kb.tsv"), 
                col.names=T, quote=F, row.names=F, sep="\t")
        
        feature_counts <- cqn.normalized[[cell_type]][feat,]
        #colnames(feature_counts) <- rownames(genotypes_filt)
    
        write.table(rownames_to_column(feature_counts, var="geneid"),
                   paste0(ct_dir,  "/", feat, "_Counts.tsv"), 
                    col.names=T, quote=F, row.names=F, sep="\t")
        gc()
    }
    
    write.table(features_to_run_ct[[cell_type]], 
           paste0("/path/to/caQTL/stuff/gwas_overlap/Matrix_QTL_Inputs/Features.",cell_type,".tsv"),
           col.names=F, quote=F, row.names=F, sep="\t")
}



# Run Matrix QTL

In [35]:
#cell_types <- c('acinar','alpha','bulk','delta','ductal','gamma')
cell_types <- c('acinar','alpha','beta','bulk','delta','ductal','gamma')
names(cell_types) <- cell_types

samples <- list()

for (cell_type in cell_types) {
    samples[[cell_type]] <- unlist(read.table(paste0('/path/to/caQTL/stuff/atac_cell_type_matrices/samples.', cell_type, '.csv'), sep=','))
}

summary(samples)
samples

       Length Class  Mode     
acinar 27     -none- character
alpha  27     -none- character
beta   27     -none- character
bulk   27     -none- character
delta  26     -none- character
ductal 26     -none- character
gamma  26     -none- character

$acinar
    x1     x2     x3     x4     x5     x6     x7     x8     x9    x10    x11 
"R207" "R217" "R218" "R221" "R223" "R226" "R228" "R234" "R237" "R238" "R246" 
   x12    x13    x14    x15    x16    x17    x18    x19    x20    x21    x22 
"R247" "R275" "R284" "R290" "R292" "R316" "R317" "R319" "R325" "R326" "R327" 
   x23    x24    x25    x26    x27 
"R332" "R353" "R354" "R362" "R363" 

$alpha
    x1     x2     x3     x4     x5     x6     x7     x8     x9    x10    x11 
"R207" "R217" "R218" "R221" "R223" "R226" "R228" "R234" "R237" "R238" "R246" 
   x12    x13    x14    x15    x16    x17    x18    x19    x20    x21    x22 
"R247" "R275" "R284" "R290" "R292" "R316" "R317" "R319" "R325" "R326" "R327" 
   x23    x24    x25    x26    x27 
"R332" "R353" "R354" "R362" "R363" 

$beta
    x1     x2     x3     x4     x5     x6     x7     x8     x9    x10    x11 
"R207" "R217" "R218" "R221" "R223" "R226" "R228" "R234" "R237" "R238" "R246" 
   x12    x13    x14    x15    x16    x17    x18    x19    x20    x21    x22 
"R247" "R275" "R284" "R290" "R292" "R316" "R317" "R319" "R325" "R326" "R327" 
   x23    x24    x25    x26    x27 
"R332" "R353" "R354" "R362" "R363" 

$bulk
    x1     x2     x3     x4     x5     x6     x7     x8     x9    x10    x11 
"R207" "R217" "R218" "R221" "R223" "R226" "R228" "R234" "R237" "R238" "R246" 
   x12    x13    x14    x15    x16    x17    x18    x19    x20    x21    x22 
"R247" "R275" "R284" "R290" "R292" "R316" "R317" "R319" "R325" "R326" "R327" 
   x23    x24    x25    x26    x27 
"R332" "R353" "R354" "R362" "R363" 

$delta
    x1     x2     x3     x4     x5     x6     x7     x8     x9    x10    x11 
"R207" "R217" "R218" "R221" "R223" "R226" "R228" "R234" "R237" "R238" "R246" 
   x12    x13    x14    x15    x16    x17    x18    x19    x20    x21    x22 
"R247" "R275" "R284" "R290" "R292" "R316" "R317" "R319" "R326" "R327" "R332" 
   x23    x24    x25    x26 
"R353" "R354" "R362" "R363" 

$ductal
    x1     x2     x3     x4     x5     x6     x7     x8     x9    x10    x11 
"R207" "R217" "R218" "R221" "R223" "R228" "R234" "R237" "R238" "R246" "R247" 
   x12    x13    x14    x15    x16    x17    x18    x19    x20    x21    x22 
"R275" "R284" "R290" "R292" "R316" "R317" "R319" "R325" "R326" "R327" "R332" 
   x23    x24    x25    x26 
"R353" "R354" "R362" "R363" 

$gamma
    x1     x2     x3     x4     x5     x6     x7     x8     x9    x10    x11 
"R207" "R217" "R218" "R221" "R223" "R226" "R228" "R234" "R237" "R238" "R246" 
   x12    x13    x14    x15    x16    x17    x18    x19    x20    x21    x22 
"R275" "R284" "R290" "R292" "R316" "R317" "R319" "R325" "R326" "R327" "R332" 
   x23    x24    x25    x26 
"R353" "R354" "R362" "R363"

In [36]:
features_to_run_ct <- list()

for (cell_type in cell_types) {
    features_to_run_ct[[cell_type]] <- unlist(read.table(paste0("/path/to/caQTL/stuff/gwas_overlap/Matrix_QTL_Inputs/Features.",cell_type,".tsv")))
}

summary(features_to_run_ct)
length(features_to_run_ct[['acinar']])
head(features_to_run_ct[['acinar']])

       Length Class  Mode     
acinar  5153  -none- character
alpha   8207  -none- character
beta    9432  -none- character
bulk   10156  -none- character
delta   4178  -none- character
ductal  2616  -none- character
gamma   3135  -none- character

[1] 5153

V11                    V12                    V13 
"chr1:3139257-3140991" "chr1:3144228-3144677" "chr1:3162149-3162863" 
                   V14                    V15                    V16 
"chr1:3164682-3166139" "chr1:3169726-3171629" "chr1:3173026-3173397"

In [37]:
covariates_file_name <- list()
cvrt <- list()

pvOutputThreshold = 1
errorCovariance = numeric()

for (cell_type in cell_types) {
    covariates_file_name[[cell_type]] <- paste0("/path/to/caQTL/stuff/gwas_overlap/Matrix_QTL_Inputs/Covariates.",cell_type,".tsv")
    
    cvrt[[cell_type]] = SlicedData$new();
    cvrt[[cell_type]]$fileDelimiter = "\t";      # the TAB character
    cvrt[[cell_type]]$fileOmitCharacters = "NA"; # denote missing values;
    cvrt[[cell_type]]$fileSkipRows = 1;          # one row of column labels
    cvrt[[cell_type]]$fileSkipColumns = 1;       # one column of row labels
    cvrt[[cell_type]]$fileSliceSize = 2000;      # read file in pieces of 2,000 rows
    cvrt[[cell_type]]$LoadFile( covariates_file_name[[cell_type]] )
    
    cvrt[[cell_type]]
}

Rows read: 18 done.

Rows read: 15 done.

Rows read: 15 done.

Rows read: 12 done.

Rows read: 18 done.

Rows read: 17 done.

Rows read: 15 done.



In [38]:
cell_types[5:7]

delta   ductal    gamma 
 "delta" "ductal"  "gamma"

In [39]:
input_QTL_dir <- '/path/to/caQTL/stuff/gwas_overlap/Matrix_QTL_Inputs'

#for (cell_type in cell_types) {
for (cell_type in cell_types[5:7]) {
    print(paste0('Starting: ', cell_type))
    for (feat in features_to_run_ct[[cell_type]]) {
    #for (feat in 'chr18:12776439-12778058') {
        ct_dir <- paste0(input_QTL_dir,"/",cell_type)
        useModel = modelLINEAR; # modelANOVA or modelLINEAR or modelLINEAR_CROSS
        
        SNP_file_name = paste0(ct_dir, "/", feat, "_Genotypes_100kb.tsv")
        expression_file_name = paste0(ct_dir,  "/", feat, "_Counts.tsv")
        output_file_name = paste0(ct_dir,  "/", feat, "_Matrix_QTL_Results_100kb.tsv")
        
        snps = SlicedData$new();
        snps$fileDelimiter = "\t";      # the TAB character
        snps$fileOmitCharacters = "NA"; # denote missing values;
        snps$fileSkipRows = 1;          # one row of column labels
        snps$fileSkipColumns = 1;       # one column of row labels
        snps$fileSliceSize = 2000;      # read file in pieces of 2,000 rows
        snps$LoadFile( SNP_file_name )
        
        gene = SlicedData$new();
        gene$fileDelimiter = "\t";      # the TAB character
        gene$fileOmitCharacters = "NA"; # denote missing values;
        gene$fileSkipRows = 1;          # one row of column labels
        gene$fileSkipColumns = 1;       # one column of row labels
        gene$fileSliceSize = 2000;      # read file in pieces of 2,000 rows
        gene$LoadFile( expression_file_name )
        
        me = Matrix_eQTL_engine(
            snps = snps,
            gene = gene,
            cvrt = cvrt[[cell_type]],
            output_file_name = output_file_name,
            pvOutputThreshold = pvOutputThreshold,
            useModel = useModel,
            errorCovariance = errorCovariance,
            verbose = FALSE,
            pvalue.hist = TRUE,
            min.pv.by.genesnp = FALSE,
            noFDRsaveMemory = FALSE)
    }
}

[1] "Starting: delta"


Rows read: 566 done.

Rows read: 1 done.

100.00% done, 566 eQTLs

Rows read: 561 done.

Rows read: 1 done.

100.00% done, 561 eQTLs

Rows read: 586 done.

Rows read: 1 done.

100.00% done, 586 eQTLs

Rows read: 588 done.

Rows read: 1 done.

100.00% done, 588 eQTLs

Rows read: 604 done.

Rows read: 1 done.

100.00% done, 604 eQTLs

Rows read: 612 done.

Rows read: 1 done.

100.00% done, 612 eQTLs

Rows read: 616 done.

Rows read: 1 done.

100.00% done, 616 eQTLs

Rows read: 616 done.

Rows read: 1 done.

100.00% done, 616 eQTLs

Rows read: 605 done.

Rows read: 1 done.

100.00% done, 605 eQTLs

Rows read: 603 done.

Rows read: 1 done.

100.00% done, 603 eQTLs

Rows read: 594 done.

Rows read: 1 done.

100.00% done, 594 eQTLs

Rows read: 596 done.

Rows read: 1 done.

100.00% done, 596 eQTLs

Rows read: 605 done.

Rows read: 1 done.

100.00% done, 605 eQTLs

Rows read: 606 done.

Rows read: 1 done.

100.00% done, 606 eQTLs

Rows read: 567 done.

Rows read: 1 done.

100.00% done, 567 eQ

Rows read: 1 done.

100.00% done, 327 eQTLs

Rows read: 327 done.

Rows read: 1 done.

100.00% done, 327 eQTLs

Rows read: 526 done.

Rows read: 1 done.

100.00% done, 526 eQTLs

Rows read: 541 done.

Rows read: 1 done.

100.00% done, 541 eQTLs

Rows read: 560 done.

Rows read: 1 done.

100.00% done, 560 eQTLs

Rows read: 566 done.

Rows read: 1 done.

100.00% done, 566 eQTLs

Rows read: 567 done.

Rows read: 1 done.

100.00% done, 567 eQTLs

Rows read: 558 done.

Rows read: 1 done.

100.00% done, 558 eQTLs

Rows read: 303 done.

Rows read: 1 done.

100.00% done, 303 eQTLs

Rows read: 319 done.

Rows read: 1 done.

100.00% done, 319 eQTLs

Rows read: 335 done.

Rows read: 1 done.

100.00% done, 335 eQTLs

Rows read: 344 done.

Rows read: 1 done.

100.00% done, 344 eQTLs

Rows read: 367 done.

Rows read: 1 done.

100.00% done, 367 eQTLs

Rows read: 400 done.

Rows read: 1 done.

100.00% done, 400 eQTLs

Rows read: 422 done.

Rows read: 1 done.

100.00% done, 422 eQTLs

Rows read: 437 do

100.00% done, 533 eQTLs

Rows read: 548 done.

Rows read: 1 done.

100.00% done, 548 eQTLs

Rows read: 619 done.

Rows read: 1 done.

100.00% done, 619 eQTLs

Rows read: 632 done.

Rows read: 1 done.

100.00% done, 632 eQTLs

Rows read: 636 done.

Rows read: 1 done.

100.00% done, 636 eQTLs

Rows read: 658 done.

Rows read: 1 done.

100.00% done, 658 eQTLs

Rows read: 481 done.

Rows read: 1 done.

100.00% done, 481 eQTLs

Rows read: 479 done.

Rows read: 1 done.

100.00% done, 479 eQTLs

Rows read: 431 done.

Rows read: 1 done.

100.00% done, 431 eQTLs

Rows read: 414 done.

Rows read: 1 done.

100.00% done, 414 eQTLs

Rows read: 411 done.

Rows read: 1 done.

100.00% done, 411 eQTLs

Rows read: 344 done.

Rows read: 1 done.

100.00% done, 344 eQTLs

Rows read: 363 done.

Rows read: 1 done.

100.00% done, 363 eQTLs

Rows read: 356 done.

Rows read: 1 done.

100.00% done, 356 eQTLs

Rows read: 351 done.

Rows read: 1 done.

100.00% done, 351 eQTLs

Rows read: 364 done.

Rows read: 1 do

Rows read: 363 done.

Rows read: 1 done.

100.00% done, 363 eQTLs

Rows read: 361 done.

Rows read: 1 done.

100.00% done, 361 eQTLs

Rows read: 428 done.

Rows read: 1 done.

100.00% done, 428 eQTLs

Rows read: 432 done.

Rows read: 1 done.

100.00% done, 432 eQTLs

Rows read: 449 done.

Rows read: 1 done.

100.00% done, 449 eQTLs

Rows read: 553 done.

Rows read: 1 done.

100.00% done, 553 eQTLs

Rows read: 558 done.

Rows read: 1 done.

100.00% done, 558 eQTLs

Rows read: 298 done.

Rows read: 1 done.

100.00% done, 298 eQTLs

Rows read: 298 done.

Rows read: 1 done.

100.00% done, 298 eQTLs

Rows read: 299 done.

Rows read: 1 done.

100.00% done, 299 eQTLs

Rows read: 292 done.

Rows read: 1 done.

100.00% done, 292 eQTLs

Rows read: 293 done.

Rows read: 1 done.

100.00% done, 293 eQTLs

Rows read: 301 done.

Rows read: 1 done.

100.00% done, 301 eQTLs

Rows read: 305 done.

Rows read: 1 done.

100.00% done, 305 eQTLs

Rows read: 308 done.

Rows read: 1 done.

100.00% done, 308 eQ

...

Rows read: 1 done.

100.00% done, 308 eQTLs

Rows read: 309 done.

Rows read: 1 done.

100.00% done, 309 eQTLs

Rows read: 331 done.

Rows read: 1 done.

100.00% done, 331 eQTLs

Rows read: 315 done.

Rows read: 1 done.

100.00% done, 315 eQTLs

Rows read: 317 done.

Rows read: 1 done.

100.00% done, 317 eQTLs

Rows read: 325 done.

Rows read: 1 done.

100.00% done, 325 eQTLs

Rows read: 379 done.

Rows read: 1 done.

100.00% done, 379 eQTLs

Rows read: 377 done.

Rows read: 1 done.

100.00% done, 377 eQTLs

Rows read: 419 done.

Rows read: 1 done.

100.00% done, 419 eQTLs

Rows read: 283 done.

Rows read: 1 done.

100.00% done, 283 eQTLs

Rows read: 300 done.

Rows read: 1 done.

100.00% done, 300 eQTLs

Rows read: 298 done.

Rows read: 1 done.

100.00% done, 298 eQTLs

Rows read: 299 done.

Rows read: 1 done.

100.00% done, 299 eQTLs

Rows read: 315 done.

Rows read: 1 done.

100.00% done, 315 eQTLs

Rows read: 317 done.

Rows read: 1 done.

100.00% done, 317 eQTLs

Rows read: 315 do

100.00% done, 316 eQTLs

Rows read: 317 done.

Rows read: 1 done.

100.00% done, 317 eQTLs

Rows read: 344 done.

Rows read: 1 done.

100.00% done, 344 eQTLs

Rows read: 344 done.

Rows read: 1 done.

100.00% done, 344 eQTLs

Rows read: 348 done.

Rows read: 1 done.

100.00% done, 348 eQTLs

Rows read: 343 done.

Rows read: 1 done.

100.00% done, 343 eQTLs

Rows read: 341 done.

Rows read: 1 done.

100.00% done, 341 eQTLs

Rows read: 350 done.

Rows read: 1 done.

100.00% done, 350 eQTLs

Rows read: 350 done.

Rows read: 1 done.

100.00% done, 350 eQTLs

Rows read: 674 done.

Rows read: 1 done.

100.00% done, 674 eQTLs

Rows read: 642 done.

Rows read: 1 done.

100.00% done, 642 eQTLs

Rows read: 646 done.

Rows read: 1 done.

100.00% done, 646 eQTLs

Rows read: 677 done.

Rows read: 1 done.

100.00% done, 677 eQTLs

Rows read: 671 done.

Rows read: 1 done.

100.00% done, 671 eQTLs

Rows read: 663 done.

Rows read: 1 done.

100.00% done, 663 eQTLs

Rows read: 654 done.

Rows read: 1 do

Rows read: 685 done.

Rows read: 1 done.

100.00% done, 685 eQTLs

Rows read: 678 done.

Rows read: 1 done.

100.00% done, 678 eQTLs

Rows read: 641 done.

Rows read: 1 done.

100.00% done, 641 eQTLs

Rows read: 616 done.

Rows read: 1 done.

100.00% done, 616 eQTLs

Rows read: 619 done.

Rows read: 1 done.

100.00% done, 619 eQTLs

Rows read: 608 done.

Rows read: 1 done.

100.00% done, 608 eQTLs

Rows read: 603 done.

Rows read: 1 done.

100.00% done, 603 eQTLs

Rows read: 604 done.

Rows read: 1 done.

100.00% done, 604 eQTLs

Rows read: 590 done.

Rows read: 1 done.

100.00% done, 590 eQTLs

Rows read: 581 done.

Rows read: 1 done.

100.00% done, 581 eQTLs

Rows read: 353 done.

Rows read: 1 done.

100.00% done, 353 eQTLs

Rows read: 357 done.

Rows read: 1 done.

100.00% done, 357 eQTLs

Rows read: 364 done.

Rows read: 1 done.

100.00% done, 364 eQTLs

Rows read: 361 done.

Rows read: 1 done.

100.00% done, 361 eQTLs

Rows read: 373 done.

Rows read: 1 done.

100.00% done, 373 eQ

Rows read: 1 done.

100.00% done, 220 eQTLs

Rows read: 211 done.

Rows read: 1 done.

100.00% done, 211 eQTLs

Rows read: 209 done.

Rows read: 1 done.

100.00% done, 209 eQTLs

Rows read: 209 done.

Rows read: 1 done.

100.00% done, 209 eQTLs

Rows read: 210 done.

Rows read: 1 done.

100.00% done, 210 eQTLs

Rows read: 209 done.

Rows read: 1 done.

100.00% done, 209 eQTLs

Rows read: 206 done.

Rows read: 1 done.

100.00% done, 206 eQTLs

Rows read: 202 done.

Rows read: 1 done.

100.00% done, 202 eQTLs

Rows read: 202 done.

Rows read: 1 done.

100.00% done, 202 eQTLs

Rows read: 195 done.

Rows read: 1 done.

100.00% done, 195 eQTLs

Rows read: 199 done.

Rows read: 1 done.

100.00% done, 199 eQTLs

Rows read: 196 done.

Rows read: 1 done.

100.00% done, 196 eQTLs

Rows read: 427 done.

Rows read: 1 done.

100.00% done, 427 eQTLs

Rows read: 463 done.

Rows read: 1 done.

100.00% done, 463 eQTLs

Rows read: 230 done.

Rows read: 1 done.

100.00% done, 230 eQTLs

Rows read: 253 do

100.00% done, 556 eQTLs

Rows read: 554 done.

Rows read: 1 done.

100.00% done, 554 eQTLs

Rows read: 574 done.

Rows read: 1 done.

100.00% done, 574 eQTLs

Rows read: 541 done.

Rows read: 1 done.

100.00% done, 541 eQTLs

Rows read: 525 done.

Rows read: 1 done.

100.00% done, 525 eQTLs

Rows read: 514 done.

Rows read: 1 done.

100.00% done, 514 eQTLs

Rows read: 447 done.

Rows read: 1 done.

100.00% done, 447 eQTLs

Rows read: 439 done.

Rows read: 1 done.

100.00% done, 439 eQTLs

Rows read: 397 done.

Rows read: 1 done.

100.00% done, 397 eQTLs

Rows read: 337 done.

Rows read: 1 done.

100.00% done, 337 eQTLs

Rows read: 335 done.

Rows read: 1 done.

100.00% done, 335 eQTLs

Rows read: 313 done.

Rows read: 1 done.

100.00% done, 313 eQTLs

Rows read: 347 done.

Rows read: 1 done.

100.00% done, 347 eQTLs

Rows read: 353 done.

Rows read: 1 done.

100.00% done, 353 eQTLs

Rows read: 346 done.

Rows read: 1 done.

100.00% done, 346 eQTLs

Rows read: 343 done.

Rows read: 1 do

# Read in results and join

In [40]:
input_QTL_dir <- '/path/to/caQTL/stuff/gwas_overlap/Matrix_QTL_Inputs'

full.results <- list()

for (cell_type in cell_types) {
    ct_dir <- paste0(input_QTL_dir,"/",cell_type)
    full.results[[cell_type]] <- data.frame()
    
    for (feat in features_to_run_ct[[cell_type]]) {
        full.results[[cell_type]] <- rbind(full.results[[cell_type]], read.table(paste0(ct_dir,  "/", feat, "_Matrix_QTL_Results_100kb.tsv"), header=T))
    }
    
    write.table(full.results[[cell_type]], paste0(input_QTL_dir,  "/Full_Matrix_QTL_Results_", cell_type,".tsv"))
}

summary(full.results)
dim(full.results[['acinar']])
head(full.results[['acinar']])

       Length Class      Mode
acinar 6      data.frame list
alpha  6      data.frame list
beta   6      data.frame list
bulk   6      data.frame list
delta  6      data.frame list
ductal 6      data.frame list
gamma  6      data.frame list

[1] 2235085       6

,SNP,gene,beta,t.stat,p.value,FDR
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,chr1:3199797:T:G,chr1:3139257-3140991,0.5862845,4.396149,0.003171636,0.2825652
2,chr1:3165198:T:C,chr1:3139257-3140991,1.0786169,3.827289,0.006480579,0.2825652
3,chr1:3167371:C:T,chr1:3139257-3140991,1.0786169,3.827289,0.006480579,0.2825652
4,chr1:3174875:C:T,chr1:3139257-3140991,1.0786169,3.827289,0.006480579,0.2825652
5,chr1:3176611:A:G,chr1:3139257-3140991,1.0786169,3.827289,0.006480579,0.2825652
6,chr1:3177326:G:A,chr1:3139257-3140991,1.0786169,3.827289,0.006480579,0.2825652


In [41]:
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,25993598,1388.3,60699594,3241.8,60699594,3241.8
Vcells,1333391298,10173.0,2310033954,17624.2,2310033954,17624.2


In [42]:
res_all <- bind_rows(full.results, .id = "cell")

dim(res_all)
head(res_all)

[1] 18542557        7

,cell,SNP,gene,beta,t.stat,p.value,FDR
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,acinar,chr1:3199797:T:G,chr1:3139257-3140991,0.5862845,4.396149,0.003171636,0.2825652
2,acinar,chr1:3165198:T:C,chr1:3139257-3140991,1.0786169,3.827289,0.006480579,0.2825652
3,acinar,chr1:3167371:C:T,chr1:3139257-3140991,1.0786169,3.827289,0.006480579,0.2825652
4,acinar,chr1:3174875:C:T,chr1:3139257-3140991,1.0786169,3.827289,0.006480579,0.2825652
5,acinar,chr1:3176611:A:G,chr1:3139257-3140991,1.0786169,3.827289,0.006480579,0.2825652
6,acinar,chr1:3177326:G:A,chr1:3139257-3140991,1.0786169,3.827289,0.006480579,0.2825652


In [43]:
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,25994448,1388.3,60699594,3241.8,60699594,3241.8
Vcells,1463191124,11163.3,2310033954,17624.2,2310033954,17624.2


In [44]:
write.table(res_all, paste0(input_QTL_dir,  "/Full_Matrix_QTL_Results.tsv"))

In [45]:
tail(res_all)

,cell,SNP,gene,beta,t.stat,p.value,FDR
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
18542552,gamma,chr9:136745581:C:T,chr9:136646051-136646757,0.03276392,0.03906384,0.9696923,0.9795384
18542553,gamma,chr9:136550111:C:T,chr9:136646051-136646757,0.03257980,0.03872040,0.9699586,0.9795384
18542554,gamma,chr9:136546400:A:G,chr9:136646051-136646757,0.01071151,0.02254376,0.9825061,0.9844610
18542555,gamma,chr9:136591998:T:C,chr9:136646051-136646757,-0.01079292,-0.02002414,0.9844610,0.9844610
18542556,gamma,chr9:136592640:C:T,chr9:136646051-136646757,-0.01079292,-0.02002414,0.9844610,0.9844610
18542557,gamma,chr9:136594440:C:T,chr9:136646051-136646757,-0.01079292,-0.02002414,0.9844610,0.9844610
